# Submission Systems 
Submission system play an important role, if you want to develop your pygromos code. Many times, they are hidden in the Simulation_runner blocks. But maybe you want to develop something, where you need direct access on the submission system? 

This notebook will give you some examples, how you can use the submission systems.
Note that all submission systems are write in the same ways, such you can exchange them quickly.

In [1]:
from pygromos.simulations.hpc_queuing.submission_systems import local # this executes your code in your local session.
from pygromos.simulations.hpc_queuing.submission_systems import lsf # this module can be used to submit to the lsf-queue (e.g. on euler)
from pygromos.simulations.hpc_queuing.submission_systems import dummy # this is a dummy system, that only prints the commands

## Local Submission

This system executes the commands directly in your current session. This allows you to locally test or execute your code. Maybe if your process needs much more time, you want later to switch to a submission system for job-queueing.

In [2]:
sub_local = local.LOCAL()
sub_local.verbose = True

In [3]:
bash_command = "sleep 2; echo \"WUHA\"; sleep 2"

job_id = sub_local.submit_to_queue(bash_command)
job_id

Submission Command: 	 s l e e p   2 ;   e c h o   " W U H A " ;   s l e e p   2
STDOUT: 
		b'WUHA\n'
END



0

In [4]:
#This is a dummy function, to not break the code!
sub_local.get_jobs_from_queue("FUN")

Searching ID:  FUN


/cluster/home/bschroed/code/pygromos/pygromos/hpc_queuing/submission_systems/local.py:159: UserWarning: Queue search was called, but no queue present!
  warnings.warn("Queue search was called, but no queue present!")


[]

## LSF Submission

The Lsf submission system allows to submit jobs to the IBM LSF-Queueing system.

**Careful! This part requires a running LSF-Queueing System on your System**

You can submit and kill jobs and arrays to the queue, as well as getting information from the queuing list.

In [5]:
#Construct system:
sub_lsf = lsf.LSF(nmpi=1, job_duration = "24:00", max_storage=100)
sub_lsf.verbose = True

sub_lsf._refresh_job_queue_list_all_s = 0 #you must wait at least 1s to update job_queue list

### Queue Checking:

In [14]:
sub_lsf.get_queued_jobs()
sub_lsf.job_queue_list

Skipping refresh of job list, as the last update is 0:00:00.005036s ago


,JOBID,USER,STAT,QUEUE,FROM_HOST,EXEC_HOST,JOB_NAME,SUBMIT_TIME
173367408,173367408,bschroed,RUN,normal.24h,eu-g1-033-1,eu-ms-001-21,Test1,2021-05-26 10:15:00
173367453,173367453,bschroed,PEND,normal.24h,eu-g1-033-1,-,Test0,2021-05-26 10:15:00
173367455,173367455,bschroed,PEND,normal.24h,eu-g1-033-1,-,Test1,2021-05-26 10:15:00
173367459,173367459,bschroed,EXIT,normal.24h,eu-g1-033-1,-,Test2,2021-05-26 10:15:00


### Submission:
here you can submit jobs to the queue as bash commands

In [7]:
bash_command = "sleep 5; echo \"WUHA\"; sleep 2"
job_name = "Test1"

job_id = sub_lsf.submit_to_queue(command=bash_command, jobName=job_name)

writing tmp-submission-file to:  ./job_Test1.sh
Submission Command: 	 bsub -JTest1 -W 24:00 -o Test1.out -n 1 -R rusage[mem=100] ./job_Test1.sh
process returned id: 173367408


In [8]:
#search for the just submitted job in the queue
sub_lsf.search_queue_for_jobid(job_id)

,JOBID,USER,STAT,QUEUE,FROM_HOST,EXEC_HOST,JOB_NAME,SUBMIT_TIME
173367408,173367408,bschroed,PEND,normal.24h,eu-g1-033-1,-,Test1,2021-05-26 10:15:00


In [9]:
sub_lsf.search_queue_for_jobname("Test1")

,JOBID,USER,STAT,QUEUE,FROM_HOST,EXEC_HOST,JOB_NAME,SUBMIT_TIME
173367408,173367408,bschroed,PEND,normal.24h,eu-g1-033-1,-,Test1,2021-05-26 10:15:00


### Submitting multiple jobs

In [10]:
bash_command = "sleep 2; echo \"WUHA\"; sleep 2"
job_ids = []
for test in range(3):
    job_name = "Test"+str(test)

    job_id = sub_lsf.submit_to_queue(command=bash_command, jobName=job_name)
    job_ids.append(job_id)

writing tmp-submission-file to:  ./job_Test0.sh
Submission Command: 	 bsub -JTest0 -W 24:00 -o Test0.out -n 1 -R rusage[mem=100] ./job_Test0.sh
process returned id: 173367453
writing tmp-submission-file to:  ./job_Test1.sh
Submission Command: 	 bsub -JTest1 -W 24:00 -o Test1.out -n 1 -R rusage[mem=100] ./job_Test1.sh
process returned id: 173367455
writing tmp-submission-file to:  ./job_Test2.sh
Submission Command: 	 bsub -JTest2 -W 24:00 -o Test2.out -n 1 -R rusage[mem=100] ./job_Test2.sh
process returned id: 173367459


In [11]:
sub_lsf.search_queue_for_jobname("Te", regex=True)

,JOBID,USER,STAT,QUEUE,FROM_HOST,EXEC_HOST,JOB_NAME,SUBMIT_TIME
173367408,173367408,bschroed,PEND,normal.24h,eu-g1-033-1,-,Test1,2021-05-26 10:15:00
173367453,173367453,bschroed,PEND,normal.24h,eu-g1-033-1,-,Test0,2021-05-26 10:15:00
173367455,173367455,bschroed,PEND,normal.24h,eu-g1-033-1,-,Test1,2021-05-26 10:15:00
173367459,173367459,bschroed,PEND,normal.24h,eu-g1-033-1,-,Test2,2021-05-26 10:15:00


### Killing a jobs

Remove a job the job queue

In [12]:
sub_lsf.kill_jobs(job_ids=[job_id])

Stopping: 173367459


In [13]:
sub_lsf.search_queue_for_jobname("Te", regex=True)

,JOBID,USER,STAT,QUEUE,FROM_HOST,EXEC_HOST,JOB_NAME,SUBMIT_TIME
173367408,173367408,bschroed,RUN,normal.24h,eu-g1-033-1,eu-ms-001-21,Test1,2021-05-26 10:15:00
173367453,173367453,bschroed,PEND,normal.24h,eu-g1-033-1,-,Test0,2021-05-26 10:15:00
173367455,173367455,bschroed,PEND,normal.24h,eu-g1-033-1,-,Test1,2021-05-26 10:15:00
173367459,173367459,bschroed,PEND,normal.24h,eu-g1-033-1,-,Test2,2021-05-26 10:15:00
